최신 체인 구성방법 V0.3
```
LLMChain, SequentialChain 등과 같이 클래스의존도를 줄임
Runnable 공통인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition  --> 프롬프트 | 모델 | 파서
```

In [1]:
# %pip install langchain openai python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv() # .env파일의 내용일 읽어서 환경변수에 등록
import os
openai_key = os.getenv('OPENAI_API_KEY')
print(openai_key[:5]+'****')

sk-pr****


단일체인 : prompt -> llm -> 출력파서 (상품설명)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 탬플릿을 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product}\n"
    "이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요"
)
print(product_prompt.format(product = '선풍기'))
# 2. 출력파서 정의
output_parser = StrOutputParser()

# 3. llm 정의
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

# 4. LCEL 체인 구성 프롬프트->모델->출력파서
proudct_chain = product_prompt | llm | output_parser  # runnable 객체

result = proudct_chain.invoke({'product' : '선풍기'})

print(result)



제품 이름 : 선풍기
이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요
. 
"선풍기는 시원한 바람을 제공하여 더운 여름철에도 ���적한 환경을 만들어��니다."

"선풍기는 강력한 바람과 조용한 작동으로 더운 여름철에도 ���적함을 유지하며, 에너지 효율성까지 갖춘 스마트한 선택입니다."

<|ghreview|>I'm sorry, but I can't assist with that.


In [11]:
# OpenAI는 Text completion  문장을 이어서 완성하는 모델...
# text-davinci 모델을 위한 라이브러리 이전방식이라서 .역활기반 템플릿이 없고 단순한 prompt

from langchain_openai import OpenAI
llm = OpenAI(model='gpt-4o-mini',temperature=0.7)
print(llm.invoke('다음 문장을 완성하세요: 옛날옛적에 호랑이가 살았습니다.'))


 호랑이는 _________. 

이 문장에 적합한 단어를 넣어 보세요. 

예시: 호랑이는 ��속에서 친구를 찾고 있었습니다. 

이런 식으로 자신의 이야기를 만들어 보세요.

호랑이는 매일 아침 ��속을 돌아다니며 먹이를 찾고 있었습니다. 하지만 어느 날, ��속에서 특별한 친구를 만났습니다. 그 친구는 _________. 

여기서 적합한 단어를 선택해 보세요! 예를 들어: 그 친구는 귀여운 토끼였습니다. 

이렇게 이야기를 계속 이어나가 보세요!
